In [1]:
# suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_rows = 250
import numpy as np
import csv
import glob
import random
import ntpath
import os
from functools import reduce

from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

## Majority Voting

### Load Train Data

In [2]:
# load stacking files
metadata_train = pd.read_csv(r'out/predictions_metadata_dev.csv', index_col = 0, header = 0)

textual_train = pd.read_csv(r'out/text_yhat_train.csv', index_col = 10, header = 0)
textual_train.index.names = ['movie']
textual_train = textual_train.drop('goodforairplanes', axis='columns')

visual_train = pd.read_csv(r'out/visual_predictions_train.csv', index_col = 0, header = 0)
#visual_train = visual_train.drop('goodforairplanes', axis='columns')

audio_train = pd.read_csv(r'out/audio_yhat_train.csv', index_col = 10, header = 0)
audio_train.index.names = ['movie']

# merge dataframes
#dfs = [metadata_train, textual_train, visual_train, audio_train]
dfs = [metadata_train, visual_train]
majority_train = reduce(lambda left, right: pd.merge(left, right, on = 'movie'), dfs)

# rearrange columns
majority_train = majority_train[[c for c in majority_train if c not in ['goodforairplanes']] + ['goodforairplanes']]
majority_train.head()

,decision_tree,knn,nearest_mean,logistic_regression,svm,bagging,random_forest,adaboost,gradient_boost,KNN,Decision Tree,Logistic Regression,SVM,Random Forest,AdaBoost,Gradient Boosting Tree,goodforairplanes
movie,,,,,,,,,,,,,,,,,
Seventh Son,0,1,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1
Welcome to Me,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,0
The Judge,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0
Transformers Age of Extinction,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0
The Normal Heart,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,1


### Load Test data

In [3]:
# load stacking files
metadata_test = pd.read_csv(r'out/predictions_metadata_test.csv', index_col = 0, header = 0)
metadata_test.index.names = ['movie']

textual_test = pd.read_csv(r'out/text_yhat_test.csv', index_col = 10, header = 0)
textual_test.index.names = ['movie']
textual_test = textual_test.drop('goodforairplanes', axis='columns')

visual_test = pd.read_csv(r'out/visual_predictions_test.csv', index_col = 0, header = 0)
#visual_test = visual_test.drop('goodforairplanes', axis='columns')

audio_test = pd.read_csv(r'out/audio_yhat_test.csv', index_col = 10, header = 0)
audio_test.index.names = ['movie']

# merge dataframes
#dfs = [metadata_test, textual_test, visual_test, audio_test]
dfs = [metadata_test, visual_test]
majority_test = reduce(lambda left, right: pd.merge(left, right, on = 'movie'), dfs)

# rearrange columns
majority_test = majority_test[[c for c in majority_test if c not in ['goodforairplanes']] + ['goodforairplanes']]
majority_test.head()

,decision_tree,knn,nearest_mean,logistic_regression,svm,bagging,random_forest,adaboost,gradient_boost,KNN,Decision Tree,Logistic Regression,SVM,Random Forest,AdaBoost,Gradient Boosting Tree,goodforairplanes
movie,,,,,,,,,,,,,,,,,
10.000 Km,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
12 Years a Slave,0,1,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1
21 Jump Street,0,1,1,1,1,0,0,0,0,0,0,1,1,0,0,0,1
2 States,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,0,1
Aanmodderfakker,1,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1


### Define Functions

In [4]:
def find_majority(li):
    
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    
    for element in li:
        
        if element in myMap: 
            myMap[element] += 1
            
        else: 
            myMap[element] = 1

        # Keep track of maximum on the go
        if myMap[element] > maximum[1]: maximum = (element, myMap[element])

    return maximum[0]

In [5]:
def majority_voting(predictions_df):
    
    X = predictions_df.iloc[:, :-1]
    y = predictions_df.iloc[:,-1]

    predictions = []

    for i in range(0,len(X.index)):

        li = X.iloc[i].tolist()
        predictions.append(find_majority(li))

    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    f1 = f1_score(y, predictions)

    scores = {"Precision": precision, "Recall": recall, "F1": f1}
        
    return scores

### Majority Voting on the Train Set
We apply Majority Voting on the training set without using cross-validation because cv would not make much sense here.

In [6]:
majority_scores_train = majority_voting(majority_train)
majority_scores_train

{'Precision': 0.5882352941176471,
 'Recall': 0.8163265306122449,
 'F1': 0.6837606837606838}

### Majority Voting on the Test Set

In [7]:
majority_scores_test = majority_voting(majority_test)
majority_scores_test

{'Precision': 0.580952380952381,
 'Recall': 0.7176470588235294,
 'F1': 0.6421052631578948}

## Label Stacking
We can reuse the train and test data from Majority Voting

In [8]:
def label_stacking(df_train, df_test = None, CV = True):

    log_reg = LogisticRegression(solver='lbfgs')  # solver='lbfgs' -> for not getting warnings

    if CV:
        X, y = df_train.iloc[:, :-1], df_train.iloc[:,-1]

        cv_scores = cross_validate(log_reg, X, y, cv=10, scoring=['precision', 'recall', 'f1'])

        precision = np.mean(cv_scores['test_precision'])
        recall = np.mean(cv_scores['test_recall'])
        f1 = np.mean(cv_scores['test_f1'])
        
    else:
        X_train, y_train = df_train.iloc[:, :-1], df_train.iloc[:,-1]
        X_test, y_test = df_test.iloc[:, :-1], df_test.iloc[:,-1]

        log_reg.fit(X_train, y_train)
        predictions = log_reg.predict(X_test)

        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)
        f1 = f1_score(y_test, predictions)

    scores = {"Precision": precision, "Recall": recall, "F1": f1}
        
    return scores

### Lable Stacking on the Train Set with CV

In [9]:
np.random.seed(1)
label_stacking_scores_train = label_stacking(majority_train)
label_stacking_scores_train

{'Precision': 0.6422619047619047, 'Recall': 0.67, 'F1': 0.638028638028638}

### Label Stacking on the Test Set

In [10]:
np.random.seed(1)
label_stacking_scores_test = label_stacking(majority_train, majority_test, CV = False)
label_stacking_scores_test

{'Precision': 0.5344827586206896,
 'Recall': 0.36470588235294116,
 'F1': 0.4335664335664336}

## Label-Feature Stacking

### Load Features Data and Merge with Train Data

In [11]:
# load stacking files
metadata_feature_train = pd.read_csv(r'out/meta_train.csv', index_col = 0, header = 0)

textual_feature_train = pd.read_csv(r'out/text_yhat_train.csv', index_col = 10, header = 0)
textual_feature_train.index.names = ['movie']
textual_feature_train = textual_train.drop('goodforairplanes', axis='columns')

visual_feature_train = pd.read_csv(r'out/visual_predictions_labels_train.csv', index_col = 0, header = 0)

audio_feature_train = pd.read_csv(r'out/audio_yhat_train.csv', index_col = 10, header = 0)
audio_feature_train.index.names = ['movie']

# merge dataframes
#dfs = [metadata_train, metadata_feature_train, textual_train, textual_feature_train, visual_feature_train, 
#       audio_train, audio_feature_train]
dfs = [metadata_train, metadata_feature_train, visual_feature_train]
features_train = reduce(lambda left, right: pd.merge(left, right, on = 'movie'), dfs)

# rearrange columns
features_train = features_train[[c for c in features_train if c not in ['goodforairplanes']] + ['goodforairplanes']]
features_train.head()

,decision_tree,knn,nearest_mean,logistic_regression,svm,bagging,random_forest,adaboost,gradient_boost,country_Australia,...,824,825,KNN,Decision Tree,Logistic Regression,SVM,Random Forest,AdaBoost,Gradient Boosting Tree,goodforairplanes
movie,,,,,,,,,,,,,,,,,,,,,
Seventh Son,0,1,0,0,1,0,0,0,0,0,...,26201.0,14542.0,1,1,1,1,1,1,1,1
Welcome to Me,1,0,1,1,1,1,1,1,1,0,...,32531.0,13753.0,0,1,0,1,1,1,0,0
The Judge,1,1,0,1,1,1,1,1,1,0,...,230400.0,119950.0,1,1,1,1,1,1,1,0
The Normal Heart,1,1,1,1,1,1,1,1,1,0,...,35320.0,20831.0,0,1,1,1,1,0,1,1
The Phantom Tollbooth,0,1,1,1,1,1,1,1,1,0,...,73984.0,38355.0,0,1,1,1,0,1,1,1


### Load Features Data and Merge with Test Data

In [12]:
# load stacking files
metadata_feature_test = pd.read_csv(r'out/meta_test.csv', index_col = 0, header = 0)

textual_feature_test = pd.read_csv(r'out/text_yhat_test.csv', index_col = 10, header = 0)
textual_feature_test.index.names = ['movie']
textual_feature_test = textual_test.drop('goodforairplanes', axis='columns')

visual_feature_test = pd.read_csv(r'out/visual_predictions_labels_test.csv', index_col = 0, header = 0)

audio_feature_test = pd.read_csv(r'out/audio_yhat_test.csv', index_col = 10, header = 0)
audio_feature_test.index.names = ['movie']

# merge dataframes
#dfs = [metadata_test, metadata_feature_test, textual_test, textual_feature_test, visual_feature_test, 
#       audio_test, audio_feature_test]
dfs = [metadata_test, metadata_feature_test, visual_feature_test]
features_test = reduce(lambda left, right: pd.merge(left, right, on = 'movie'), dfs)

# rearrange columns
features_test = features_test[[c for c in features_test if c not in ['goodforairplanes']] + ['goodforairplanes']]
features_test.head()

,decision_tree,knn,nearest_mean,logistic_regression,svm,bagging,random_forest,adaboost,gradient_boost,country_Australia,...,824,825,KNN,Decision Tree,Logistic Regression,SVM,Random Forest,AdaBoost,Gradient Boosting Tree,goodforairplanes
movie,,,,,,,,,,,,,,,,,,,,,
10.000 Km,1,0,1,1,1,1,1,1,1,0,...,518400.0,269700.00,1,1,1,1,1,1,1,1
12 Years a Slave,0,1,0,0,1,1,0,0,0,0,...,30212.0,14583.00,0,0,1,1,1,0,0,1
21 Jump Street,0,1,1,1,1,0,0,0,0,0,...,230400.0,119790.00,0,0,1,1,0,0,0,1
2 States,1,0,0,1,1,1,1,1,1,0,...,48676.0,34515.00,0,1,0,1,1,1,0,1
Aanmodderfakker,1,0,1,1,1,1,1,1,1,0,...,2971.6,970.54,1,0,0,1,1,0,0,1


To use Labe-Feature Stacking we can use the function `label_stacking` with our new train and test sets

### Label-Feature Stacking on the Train Set with CV

In [13]:
np.random.seed(1)
label_feature_stacking_train = label_stacking(features_train)
label_feature_stacking_train

C:\Users\lilli\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\lilli\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\lilli\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\lilli\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\lilli\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

{'Precision': 0.5802380952380952, 'Recall': 0.79, 'F1': 0.6647435897435896}

## Compare the Results to the Paper

In [14]:
voting_cv = {"Precision": 0.94, "Recall": 0.57, "F1": 0.71}
label_stacking_cv = {"Precision": 0.72, "Recall": 0.86, "F1": 0.78}
label_attribute_stacking_cv = {"Precision": 0.71, "Recall": 0.79, "F1": 0.75}
voting_test = {"Precision": 0.62, "Recall": 0.80, "F1": 0.70}
label_stacking_test = {"Precision": 0.62, "Recall": 0.90, "F1": 0.73}

In [15]:
print("MAJORITY VOTING CV")
print("Paper: ", voting_cv)
print("Group15: ", majority_scores_train)
print("---------------------------------------------------------------------------------------------------")

print("LABEL STACKING CV")
print("Paper: ", label_stacking_cv)
print("Group15: ", label_stacking_scores_train)
print("---------------------------------------------------------------------------------------------------")

print("LABEL ATTRIBUTE STACKING CV")
print("Paper: ", label_attribute_stacking_cv)
print("Group15: ", label_feature_stacking_train)
print("---------------------------------------------------------------------------------------------------")

print("MAJORITY VOTING TEST")
print("Paper: ", voting_test)
print("Group15: ", majority_scores_test)
print("---------------------------------------------------------------------------------------------------")

print("LABEL STACKING TEST")
print("Paper: ", label_stacking_test)
print("Group15: ", label_stacking_scores_test)
print("---------------------------------------------------------------------------------------------------")

MAJORITY VOTING CV
Paper:  {'Precision': 0.94, 'Recall': 0.57, 'F1': 0.71}
Group15:  {'Precision': 0.5882352941176471, 'Recall': 0.8163265306122449, 'F1': 0.6837606837606838}
---------------------------------------------------------------------------------------------------
LABEL STACKING CV
Paper:  {'Precision': 0.72, 'Recall': 0.86, 'F1': 0.78}
Group15:  {'Precision': 0.6422619047619047, 'Recall': 0.67, 'F1': 0.638028638028638}
---------------------------------------------------------------------------------------------------
LABEL ATTRIBUTE STACKING CV
Paper:  {'Precision': 0.71, 'Recall': 0.79, 'F1': 0.75}
Group15:  {'Precision': 0.5802380952380952, 'Recall': 0.79, 'F1': 0.6647435897435896}
---------------------------------------------------------------------------------------------------
MAJORITY VOTING TEST
Paper:  {'Precision': 0.62, 'Recall': 0.8, 'F1': 0.7}
Group15:  {'Precision': 0.580952380952381, 'Recall': 0.7176470588235294, 'F1': 0.6421052631578948}
----------------------